## **1. Inisialisasi dan Konfigurasi Pipeline**

Langkah awal ini adalah fondasi dari keseluruhan pipeline. Di sini, kita melakukan beberapa hal penting:
* **Impor Pustaka**: Mengimpor semua modul yang diperlukan dari TensorFlow (tf), TFX, dan pustaka lainnya. `InteractiveContext` secara khusus digunakan agar kita dapat menjalankan komponen TFX secara interaktif di dalam notebook.
* **Definisi Konstanta**: Menetapkan variabel-variabel global yang akan digunakan di seluruh pipeline. Ini termasuk nama pipeline (`PIPELINE_NAME`), lokasi penyimpanan metadata ML (`METADATA_PATH`), direktori untuk model yang siap di-*deploy* (`SERVING_MODEL_DIR`), dan lokasi data mentah (`DATA_ROOT`). Praktik ini membuat pipeline lebih terstruktur dan mudah dikelola.
* **Konteks Interaktif**: `InteractiveContext` diinisialisasi untuk mengatur lingkungan eksekusi pipeline di dalam notebook ini. Konteks ini akan melacak semua artefak (seperti data, skema, model) dan eksekusi komponen, menyimpannya dalam basis data SQLite di lokasi metadata yang telah ditentukan.

In [1]:
import os
import tensorflow as tf
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Tuner, Trainer, Evaluator, Pusher
from tfx.proto import trainer_pb2, pusher_pb2, example_gen_pb2
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel, standard_artifacts
from tfx.types.standard_artifacts import Model, ModelBlessing
import tensorflow_model_analysis as tfma

In [2]:
import pandas as pd

In [3]:
PIPELINE_NAME = "kstarid-pipeline"
SCHEMA_PIPELINE_NAME = "apple-quality-schema"

PIPELINE_ROOT = os.path.join(PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)
DATA_ROOT = 'data'

In [4]:
MODEL_NAME = "apple-quality-model"

In [5]:
serving_model_path = os.path.join(SERVING_MODEL_DIR, MODEL_NAME)

In [6]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

In [7]:
df = pd.read_csv("data/apple_quality.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   A_id         4000 non-null   int64  
 1   Size         4000 non-null   float64
 2   Weight       4000 non-null   float64
 3   Sweetness    4000 non-null   float64
 4   Crunchiness  4000 non-null   float64
 5   Juiciness    4000 non-null   float64
 6   Ripeness     4000 non-null   float64
 7   Acidity      4000 non-null   float64
 8   Quality      4000 non-null   object 
dtypes: float64(7), int64(1), object(1)
memory usage: 281.4+ KB


## **2. Penyerapan Data (Data Ingestion) dengan CsvExampleGen**

Tahap ini adalah pintu gerbang masuknya data ke dalam pipeline TFX. Komponen yang digunakan adalah **`CsvExampleGen`**.

* **Tujuan**: Membaca data dari sumber eksternal (dalam kasus ini, sebuah file CSV) dan mengubahnya menjadi format `TFRecord`. Format ini sangat efisien untuk digunakan dalam ekosistem TensorFlow.
* **Proses**:
    1.  **`input_base`**: Komponen ini diarahkan ke direktori `DATA_ROOT` ('data/') untuk menemukan file `student_habits_performance.csv`.
    2.  **`output_config`**: Kita mendefinisikan cara data akan dibagi. Di sini, data dibagi menjadi dua set: `train` dan `eval` dengan perbandingan **8:2** (ditentukan oleh `hash_buckets=8` untuk train dan `hash_buckets=2` untuk eval). Pembagian ini penting agar model dilatih pada satu set data dan dievaluasi pada set data lain yang belum pernah dilihatnya.
* **Hasil**: Komponen ini menghasilkan artefak bernama **`Examples`** yang berisi data `train` dan `eval` dalam format TFRecord. Eksekusi melalui `interactive_context.run()` menyimpan artefak ini ke dalam direktori `kstarid-pipeline/CsvExampleGen`.

In [8]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=2)
    ])
)

In [9]:
ingested_data = CsvExampleGen(input_base=DATA_ROOT, output_config=output)
interactive_context.run(ingested_data)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 13
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

## **3. Validasi Data**

Validasi data adalah langkah krusial untuk memastikan kualitas dan konsistensi data sebelum masuk ke tahap pelatihan. Proses ini terdiri dari tiga komponen utama:

#### **3.1. StatisticsGen: Menghasilkan Statistik Data**
* **Tujuan**: Menghitung statistik deskriptif untuk setiap fitur dalam dataset. Ini mencakup metrik seperti rata-rata, standar deviasi, nilai minimum/maksimum untuk fitur numerik, serta frekuensi nilai unik untuk fitur kategoris.
* **Proses**: `StatisticsGen` mengambil artefak `Examples` (dari CsvExampleGen) sebagai input dan menganalisis set data `train` dan `eval` secara terpisah.
* **Hasil**: Menghasilkan artefak `ExampleStatistics`. Saat divisualisasikan menggunakan `interactive_context.show()`, kita dapat melihat distribusi, nilai yang hilang, dan properti lainnya dari setiap fitur, yang sangat berguna untuk memahami data secara mendalam.

#### **3.2. SchemaGen: Menginferensi Skema Data**
* **Tujuan**: Membuat skema data secara otomatis berdasarkan statistik yang dihasilkan oleh `StatisticsGen`. Skema ini mendefinisikan tipe data (INT, FLOAT, STRING), valensi (apakah fitur tunggal atau list), dan domain (nilai-nilai yang diharapkan untuk fitur kategoris) dari setiap fitur.
* **Proses**: `SchemaGen` menganalisis artefak `ExampleStatistics` untuk menginferensi properti-properti ini.
* **Hasil**: Menghasilkan artefak `Schema`. Skema ini berfungsi sebagai "kontrak" atau aturan yang harus dipatuhi oleh data. Kita dapat melihat tabel yang merangkum tipe data, keberadaan (apakah wajib ada atau tidak), dan domain untuk setiap fitur.

#### **3.3. ExampleValidator: Memeriksa Anomali Data**
* **Tujuan**: Mengidentifikasi anomali atau kejanggalan dalam data dengan membandingkan statistik data saat ini dengan skema yang telah dibuat.
* **Proses**: `ExampleValidator` mencari masalah seperti:
    * Tipe data yang tidak cocok.
    * Fitur yang seharusnya ada tetapi hilang.
    * Nilai fitur kategoris yang berada di luar domain yang ditentukan.
* **Hasil**: Menghasilkan artefak `ExampleAnomalies`. Dalam kasus ini, output menunjukkan **"No anomalies found,"** yang merupakan hasil ideal. Ini menandakan bahwa data kita bersih, konsisten, dan sesuai dengan skema yang diharapkan.

In [10]:
stats = StatisticsGen(examples = ingested_data.outputs["examples"])
interactive_context.run(stats)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 14
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [11]:
interactive_context.show(stats.outputs['statistics'])

In [12]:
skema = SchemaGen(statistics=stats.outputs["statistics"])
interactive_context.run(skema)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 15
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [13]:
interactive_context.show(skema.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'A_id',INT,required,,-
'Acidity',FLOAT,required,,-
'Crunchiness',FLOAT,required,,-
'Juiciness',FLOAT,required,,-
'Quality',STRING,required,,'Quality'
'Ripeness',FLOAT,required,,-
'Size',FLOAT,required,,-
'Sweetness',FLOAT,required,,-
'Weight',FLOAT,required,,-


,Values
Domain,
'Quality',"'bad', 'good'"


In [14]:
validing = ExampleValidator(
    statistics=stats.outputs["statistics"],
    schema=skema.outputs["schema"]
)
interactive_context.run(validing)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 16
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [15]:
interactive_context.show(validing.outputs["anomalies"])

## **4. Rekayasa Fitur (Feature Engineering) dengan Transform**

Tahap ini berfokus pada pra-pemrosesan dan transformasi data agar siap digunakan oleh model.
* **Tujuan**: Menerapkan transformasi fitur secara konsisten pada data `train` dan `eval`. Komponen **`Transform`** memastikan bahwa transformasi yang sama yang diterapkan saat pelatihan juga akan diterapkan saat inferensi (prediksi), sehingga menghindari *training-serving skew*.
* **Proses**:
    1.  `Transform` memanggil fungsi `preprocessing_fn` yang didefinisikan dalam file `student_transform.py`.
    2.  Dalam `preprocessing_fn` ini, semua fitur numerik dan label (`exam_score`) dikonversi (di-*cast*) menjadi tipe data `tf.float32`. Meskipun sederhana, ini adalah contoh bagaimana transformasi yang lebih kompleks (seperti normalisasi, pembagian bucket, atau pembuatan *feature cross*) dapat diimplementasikan.
* **Hasil**: Komponen ini menghasilkan dua artefak utama:
    * **`transformed_examples`**: Data `train` dan `eval` yang telah melalui proses transformasi.
    * **`transform_graph`**: Sebuah grafik TensorFlow yang menyimpan logika transformasi. Grafik inilah yang akan digunakan kembali oleh `Trainer`, `Evaluator`, dan saat model di-*deploy* untuk memastikan konsistensi.

In [16]:
TRANSFORM_MODULE_FILE = "apple_transform.py"
transform = Transform(
    examples=ingested_data.outputs["examples"],
    schema=skema.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.


INFO:tensorflow:Assets written to: kstarid-pipeline\Transform\transform_graph\17\.temp_path\tftransform_tmp\8b745ec8b7d14a129474d440c4b77ae2\assets


INFO:tensorflow:Assets written to: kstarid-pipeline\Transform\transform_graph\17\.temp_path\tftransform_tmp\8b745ec8b7d14a129474d440c4b77ae2\assets


INFO:tensorflow:struct2tensor is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 17
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [17]:
TUNER_MODULE_FILE = "apple_tuner.py"
tuner = Tuner(
    module_file=os.path.abspath(TUNER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=skema.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=500),
    eval_args=trainer_pb2.EvalArgs(num_steps=100)
)
interactive_context.run(tuner)

Trial 30 Complete [00h 00m 34s]
val_accuracy: 0.9309375286102295

Best val_accuracy So Far: 0.9532812237739563
Total elapsed time: 00h 07m 36s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in kstarid-pipeline\.temp\18\apple_quality_tuning
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 3
units_0: 128
dropout: 0.4
learning_rate: 0.001
units_1: 96
units_2: 64
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0023
Score: 0.9532812237739563
Trial summary
Hyperparameters:
num_layers: 3
units_0: 96
dropout: 0.1
learning_rate: 0.01
units_1: 128
units_2: 256
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0020
Score: 0.9482812285423279
Trial summary
Hyperparameters:
num_layers: 2
units_0: 128
dropout: 0.30000000000000004
learning_rate: 0.01
units_1: 64
units_2: 224
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0015
Score: 0.9478124976158142
Trial summary
Hyperparameters:
num_layers: 3
units_0: 96
dropout: 0.1
learning_rate: 0.01
units_1: 128
units_2: 256
tuner/epochs: 7
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/roun

ExecutionResult(
    component_id: Tuner
    execution_id: 18
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={})
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}))

## **6. Pelatihan Model dengan Trainer**

Ini adalah inti dari proses machine learning, di mana model sebenarnya dilatih.
* **Tujuan**: Melatih model Neural Network menggunakan data yang telah ditransformasi dan hyperparameter terbaik yang ditemukan oleh `Tuner`.
* **Proses**:
    1.  Komponen **`Trainer`** mengambil `transformed_examples`, `transform_graph`, `schema`, dan `best_hyperparameters` sebagai input.
    2.  Logika pelatihan didefinisikan dalam file `student_trainer.py`. Fungsi `_build_keras_model` membangun arsitektur model berdasarkan parameter yang diterima, dan `run_fn` menjalankan proses `model.fit`.
    3.  Model dilatih selama 15 *epoch* pada data `train` dan divalidasi pada data `eval` di setiap akhir *epoch*.
* **Hasil**: Komponen ini menghasilkan artefak `Model` yang berisi model terlatih yang siap untuk dievaluasi lebih lanjut atau di-*deploy*.Trainer

In [18]:
TRAINER_MODULE_FILE = "apple_trainer.py"
trainer = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=skema.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(num_steps=500),
    eval_args=trainer_pb2.EvalArgs(num_steps=100),
)
interactive_context.run(trainer)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 A_id_xf (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 Size_xf (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 Weight_xf (InputLayer)         [(None, 1)]          0           []                               
                                                                                                  
 Sweetness_xf (InputLayer)      [(None, 1)]          0           []                               
                                                                                            

INFO:tensorflow:Assets written to: kstarid-pipeline\Trainer\model\19\Format-Serving\assets


INFO:tensorflow:Assets written to: kstarid-pipeline\Trainer\model\19\Format-Serving\assets


100/100 [==============================] - 0s 2ms/step - loss: 0.1341 - accuracy: 0.9530 - precision: 0.9374 - recall: 0.9692


ExecutionResult(
    component_id: Trainer
    execution_id: 19
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [19]:
model_resolver = Resolver(
    strategy_class=LatestBlessedModelStrategy,
    model=Channel(type=Model),
    model_blessing=Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')
interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 20
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={})
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}))

## **7. Evaluasi dan Validasi Model dengan Evaluator**

Setelah model dilatih, kita perlu mengevaluasi performanya secara mendalam.
* **Tujuan**: Menganalisis kinerja model terlatih pada set data evaluasi. **`Evaluator`** tidak hanya menghitung metrik keseluruhan, tetapi juga memungkinkan analisis pada irisan (slice) data yang berbeda.
* **Proses**:
    1.  **`Resolver`**: Pertama, `Resolver` dengan strategi `LatestBlessedModelStrategy` digunakan untuk mencari model "blessed" (model yang sudah lolos validasi sebelumnya) sebagai *baseline*. Pada eksekusi pertama, tidak ada *baseline* yang ditemukan.
    2.  **`Evaluator`**: Komponen ini mengambil model baru (dari `Trainer`) dan data evaluasi.
    3.  **`EvalConfig`**: Kita mendefinisikan konfigurasi evaluasi yang mencakup:
        * `ModelSpec` dengan label key 'Quality_xf' (label yang sudah ditransformasi)
        * `SlicingSpec` untuk evaluasi pada keseluruhan dataset
        * `MetricsSpec` untuk menghitung metrik `ExampleCount`, `BinaryAccuracy`, dan `AUC`
        * Threshold untuk `binary_accuracy` dengan batas bawah 0.8 (model dianggap 'BLESSED' jika akurasi di atas 80%)
* **Hasil**:
    * `Evaluator` menghasilkan artefak **`evaluation`** yang berisi hasil analisis metrik.
    * Komponen ini juga menghasilkan artefak **`blessing`**. `blessing` ini menentukan apakah model baru "diberkati" (dianggap cukup baik untuk produksi) atau tidak.
    * Visualisasi `render_slicing_metrics` menunjukkan performa model pada metrik accuracy, AUC, dan jumlah contoh yang dievaluasi.

In [20]:
import tensorflow_model_analysis as tfma

simple_eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='Quality_xf')
    ],
    slicing_specs=[
        tfma.SlicingSpec()
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            thresholds={
                # Metrik yang dievaluasi: accuracy
                'binary_accuracy': tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        # Model dianggap 'BLESSED' jika accuracy di atas 0.8
                        lower_bound={'value': 0.8} 
                    )
                )
            },
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='BinaryAccuracy'),
                tfma.MetricConfig(class_name='AUC')
            ]
        )
    ]
)

In [21]:
evaluator_simple = Evaluator(
    examples=transform.outputs['transformed_examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=simple_eval_config
)
interactive_context.run(evaluator_simple)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 21
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={})
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}))

In [22]:
eval_result = evaluator_simple.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

## **8. Penyiapan Model untuk Deployment (Pusher)**

Langkah terakhir dalam pipeline ini adalah memindahkan model yang telah divalidasi ke lokasi yang dapat diakses untuk inferensi atau penyajian (*serving*).
* **Tujuan**: "Mendorong" (*push*) model yang telah terbukti bagus (berdasarkan artefak `blessing`) ke direktori *deployment*.
* **Proses**:
    1.  Komponen **`Pusher`** memeriksa artefak `blessing` dari `Evaluator`.
    2.  Jika model diberkati (`blessed`), `Pusher` akan menyalin artefak model dari `Trainer` ke lokasi yang ditentukan dalam `push_destination`.
    3.  Dalam kasus ini, `push_destination` adalah direktori `serving_model/kstarid-pipeline/apple-quality-model`.
* **Hasil**: Artefak model sekarang tersedia di direktori `serving_model`. Model ini memiliki struktur yang siap untuk di-*load* oleh platform penyajian seperti **TensorFlow Serving**, sehingga dapat digunakan untuk membuat prediksi pada data baru.

In [23]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator_simple.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=serving_model_path
        )
    ))

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 22
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}))